<a href="https://colab.research.google.com/github/TheAmirHK/DantzigWolfeDecomposition/blob/main/WolfeDantzig_decomp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective Function (**Original Problem**)

$$max Z = 3x_{1,1} + 5x_{1,2} + 4x_{2,1} + 2x_{2,2} + 1x_{3,1} + 6x_{3,2}$$

### Constraints:

$$1x_{1,1} + 2x_{1,2} \leq 4$$
$$3x_{1,1} + 1x_{1,2} \leq 6$$
$$2x_{2,1} + 1x_{2,2} \leq 5$$
$$-1x_{2,1} + 2x_{2,2} \leq 7$$
$$1x_{3,1} - 1x_{3,2} \leq 3$$
$$2x_{3,1} + 3x_{3,2} \leq 8$$

$$(x_{1,1} + 2x_{1,2}) + (2x_{2,1} + 1x_{2,2}) + (1x_{3,1} + 1x_{3,2}) \leq 10$$
$$(3x_{1,1} + 1x_{1,2}) + (1x_{2,1} + 2x_{2,2}) + (2x_{3,1} + 3x_{3,2}) \leq 15$$

$$x_{i,j} \geq 0, \quad \forall i, j$$



In [ ]:
!pip install mip

In [14]:
from mip import Model, MAXIMIZE, xsum, CONTINUOUS
import numpy as np

In [15]:
# Main problem parameters
num_units = 3
num_resources = 2
num_variables_per_unit = 2

resource_limits = [10, 15]

unit_objectives = [
    np.array([3, 5]),
    np.array([4, 2]),
    np.array([1, 6])]

unit_constraints = [
    np.array([[1, 2], [3, 1]]),
    np.array([[2, 1], [-1, 2]]),
    np.array([[1, -1], [2, 3]])]

unit_rhs = [
    np.array([4, 6]),
    np.array([5, 7]),
    np.array([3, 8])]

In [26]:
# Original problem
#--------------

model = Model(sense=MAXIMIZE)

x = [[model.add_var(name=f"x_{i}_{j}", lb=0, var_type=CONTINUOUS)
      for j in range(num_variables_per_unit)] for i in range(num_units)]

model.objective = xsum(unit_objectives[i][j] * x[i][j]
                       for i in range(num_units)
                       for j in range(num_variables_per_unit))

for i in range(num_units):
    for r in range(num_resources):
        model += xsum(unit_constraints[i][r][j] * x[i][j]
                      for j in range(num_variables_per_unit)) <= unit_rhs[i][r]

for r in range(num_resources):
    model += xsum(unit_constraints[i][r][j] * x[i][j]
                  for i in range(num_units)
                  for j in range(num_variables_per_unit)) <= resource_limits[r]

model.optimize()

print("Optimization Status:", model.status)
print("Objective Value:", model.objective_value)
for i in range(num_units):
    for j in range(num_variables_per_unit):
        print(f"x_{i}_{j}: {x[i][j].x}")

Optimization Status: OptimizationStatus.OPTIMAL
Objective Value: 36.8
x_0_0: 1.6
x_0_1: 1.2
x_1_0: 2.5
x_1_1: 0.0
x_2_0: 0.0
x_2_1: 2.6666666666666665


In [28]:
# Wolfe-Dantzing decomposition
#-----------------------------

# Master problem
master_problem = Model(sense=MAXIMIZE)

lambda_vars = [master_problem.add_var(name=f"lambda_{i}", lb=0, var_type=CONTINUOUS) for i in range(num_units)]

master_problem.objective = xsum(lambda_vars[i] * unit_objectives[i].sum() for i in range(num_units))

for r in range(num_resources):
    master_problem += xsum(lambda_vars[i] * unit_constraints[i][r].sum() for i in range(num_units)) <= resource_limits[r]

master_problem.optimize()

lambda_opt = [lambda_vars[i].x for i in range(num_units)]

print("Master Problem Results:")
print(f"Objective Value: {master_problem.objective_value}")
for i in range(num_units):
    print(f"lambda_{i}:", lambda_opt[i])


# Subproblems
for i in range(num_units):
    subproblem = Model(sense=MAXIMIZE)

    x = [subproblem.add_var(name=f"x_{i}_{j}", lb=0, var_type=CONTINUOUS) for j in range(num_variables_per_unit)]

    subproblem.objective = xsum(unit_objectives[i][j] * x[j] for j in range(num_variables_per_unit))

    for r in range(num_resources):
        subproblem += xsum(unit_constraints[i][r][j] * x[j] for j in range(num_variables_per_unit)) <= unit_rhs[i][r]

    subproblem.optimize()

    print(f"\nSubproblem Unit {i} Results:")
    print(f"Objective Value: {subproblem.objective_value}")
    for j in range(num_variables_per_unit):
        print(f"x_{i}_{j}:", x[j].x)

Master Problem Results:
Objective Value: 36.333333333333336
lambda_0: 0.0
lambda_1: 3.3333333333333335
lambda_2: 2.3333333333333335

Subproblem Unit 0 Results:
Objective Value: 10.8
x_0_0: 1.6
x_0_1: 1.2

Subproblem Unit 1 Results:
Objective Value: 10.0
x_1_0: 2.5
x_1_1: 0.0

Subproblem Unit 2 Results:
Objective Value: 16.0
x_2_0: 0.0
x_2_1: 2.6666666666666665
